## Использование линейной регрессии для построения причинно-следственных выводов.

**Материал подготовлен для курса [Объяснимое машинное обучение](https://stepik.org/course/198640/info).**

Стараясь оценить связь признака с целевой переменной, исследователи часто прибегают к анализу коэффициента корреляции. Кроме того, коэффициент корреляции — это первое, с чем знакомят на курсах Data science, так что знание о нем есть даже у начинающих. Однако:

> *Correlation is not causation.*

или "корреляция не равна причинно-следственной связи". Но как быть, если причинно-следственную связь оценить всё же хочется?

Если вы проходите курс [Объяснимое машинное обучение](https://stepik.org/course/198640/info), то в блоке о линейных моделях мы с вами уже познакомились с использованием коэффициентов регрессии для анализа важности признаков. Теперь мы предлагаем изучить возможность использования коэффициентов регрессии на более глубоком уровне!

Чтобы материал был понятен каждому, начнем с самого начала.

In [9]:
#Импорт библиотек
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf


Оценим важность различных признаков для нерадивых студентиков?

**Математика**

Пусть:\
$T$ -- признак, влияние которого мы хотим оценить (готовность получить высшее образование)\
$Y$ -- значения целевой переменной без введения дополнительного признака\
$Y_0$ -- значения целевой переменной при условии, что студент не был готов получать вышку\
$Y_1$ -- значения целевой переменной при условии, что студент был готов получать вышку\
$TE$ -- амплитуда измененияя признака при изменении значения с 0 на 1

In [7]:
#В идеальном мире наш набор данных выглядел бы так
ate_att_data = pd.DataFrame(dict(
      i= [1,2,3,4],
      Y0=[500,600,800,700],
      Y1=[450,600,600,750],
      T= [0,0,1,1],
      Y= [500,600,600,750],
      TE=[-50,0,-200,50],
))
ate_att_data

,i,Y0,Y1,T,Y,TE
0,1,500,450,0,500,-50
1,2,600,600,0,600,0
2,3,800,600,1,600,-200
3,4,700,750,1,750,50


In [8]:
#Но в реальном, он выглядит так
pd.DataFrame(dict(
    i= [1,2,3,4],
    Y0=[500,600,np.nan,np.nan],
    Y1=[np.nan,np.nan,600,750],
    T= [0,0,1,1],
    Y= [500,600,600,750],
    TE=[np.nan,np.nan,np.nan,np.nan],
))

,i,Y0,Y1,T,Y,TE
0,1,500.0,NaN,0,500,NaN
1,2,600.0,NaN,0,600,NaN
2,3,NaN,600.0,1,600,NaN
3,4,NaN,750.0,1,750,NaN


Студентов у нас много. Для удобства, введем еще пару вещей:

- $Y_{i0}$ - потенциальная оценка $i-го$ студента, при неготовности получать вышку мотивации\
- $Y_{i1}$ - потенциальная оценка $i-го$ студента, при готовности получать вышку

- Тогда влияние признака "готовность получить вышку" можно определить как: $Y_{i1} - Y_{i0}$


- $ATE$ *average treatment effect* — средний эффект наличия признака\
- $ATT$ *average treatment effect on treated* — средний эффект наличия признака для наблюдений, имеющих данный признак

Заметим, что:\
$ATE = E[Y_1 - Y_0]$ — матожидание от разности целевого значения с наличием и остутствием признака\
$ATT = E[Y_1 - Y_0|T=1]$ -  матожидание от разности целевого значения с наличием и остутствием признака при условии, что признак равен 1.

По идеальному набору данных мы смогли бы рассчитать всё по определению. По реальному — нет. И так как мы решили работать с реальностью, рассмотрим:

$$E[Y|T=1] - E[Y|T=0] = E[Y_1|T=1] - E[Y_0|T=0] = E[Y_1|T=1] - E[Y_0|T=1] + E[Y_0|T=1] - E[Y_0|T=0]=E[Y_1-Y_0|T=1] + (E[Y_0|T=1] - E[Y_0|T=0])$$


Получили: $E[Y_1-Y_0|T=1] + (E[Y_0|T=1] - E[Y_0|T=0])$

Первое слагаемое $E[Y_1-Y_0|T=1]$ определяет важность признака, второе $(E[Y_0|T=1] - E[Y_0|T=0])$ — случайный шум (bias).

**Постановка проблемы:** \
Влияние некоторых переменных невозможно проверить, используя А/В тестирование. Например:

- Некоторые эксперименты неэтичны. Нельзя заставить студентов употреблять алкоголь, чтобы увидеть, как это повлияет на их успеваемость в университете.
- Проведение ряда других экспериментов запрещено юридическими ограничениями. Например, вы не можете устанавливать разные цены на один и тот же продукт.
- Также существуют ситуации, когда провести эксперимент невозможно. Если происходит ребрендинг, запускать товар на две достаточно большие непересекающися группы может быть дорого.


В этом случае исследователю приходится работать с тем, что есть.

Что может помочь оценить влияние признака? **Линейная регрессия!**

**Используемый датасет:** [данные об успеваемости студентов Калифорнийского университета в Ирвине](https://archive.ics.uci.edu/dataset/320/student+performance).


Выясним, как готовность получить высшее образование влияет на итоговый балл по математике. А именно попробуем построить модель:

$$ math_{-}grade = \beta_0+k*higher + τ,$$
где:
- $\beta_0$ – свободный член
- $higher$ – готовность получить высшее образование
- $τ$ — случайный шум

In [17]:
data = pd.read_csv('https://github.com/aiedu-courses/all_datasets/raw/main/student-mat.csv',
                   sep=';')
data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [13]:
model = smf.ols('G3 ~ higher', data=data).fit()
model.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,6.8000,1.009,6.743,0.000,4.817,8.783
higher[T.yes],3.8080,1.035,3.679,0.000,1.773,5.843


**Что значит каждый коэффициент?**
- coef - при Intercept означает среднее значение целевой переменной для всей выборки, при признаке — примерное изменение прогноза, если признак равен 1;
- std err	-- стандартное отклонение влияния (веса)
- pvalue — уровень значимости, так как он менее 0.5 здесь, то можно сказать, что готовность к высшему образованию значимо влияет на прогноз

Если бы мы работали с более простыми данными, то взятие среднего по оценке дало бы нам intecept.

In [ ]:
#Проверьте, так ли это

**Выдвиньте гипотезу, почему:**

Таким образом заметим, что данный подход ограничен наличием скрытых взаимосвязей в данных. Поэтому к любым причинно-следственным выводам, основанным на наблюдениях, следует относиться с долей скептицизма. Мы не можем быть уверены, что полученная нами оценка полностью объективна.

Имея данные наблюдений, мы можем только использовать  здравый смысл, чтобы выдвинуть гипотезы, валидные к проверке, но несмотря на ограничения, мы все равно можем использовать ряд точных инстурментов, таких как регрессия, чтобы сделать все возможное, чтобы прийти к некоторым выводам о мире.

Спасибо за ваше время на материал!

Источники:
1. [Casual Inference for the Brave and True](https://matheusfacure.github.io/python-causality-handbook/01-Introduction-To-Causality.html)
2. [Linear regressions for causal conclusions](https://towardsdatascience.com/linear-regressions-for-causal-conclusions-34c6317c5a11)
